In [1]:
!pip install backtesting pandas numpy python-binance ta

Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 25.0 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
%%writefile indicators.py
import pandas as pd
import ta

def add_indicators_15m(df):
    df = df.copy()
    df["SMA_20"] = ta.trend.sma_indicator(df["Close"], window=20)
    df["RSI_14"] = ta.momentum.rsi(df["Close"], window=14)
    return df

def add_indicators_1h(df):
    df = df.copy()
    df["SMA_50"] = ta.trend.sma_indicator(df["Close"], window=50)
    return df


Overwriting indicators.py


In [3]:
%%writefile strategy.py
class MultiTimeframeStrategy:
    """
    Stateless strategy.
    Makes decisions only.
    No order placement.
    No position tracking.
    """

    def generate_signal(self, df_15m, df_1h, in_position: bool):
        """
        Returns:
            "BUY", "SELL", or None
        """

        # Safety checks
        if len(df_15m) < 20 or len(df_1h) < 50:
            return None

        rsi_15m = df_15m["RSI_14"].iloc[-1]
        sma_50_1h = df_1h["SMA_50"].iloc[-1]
        close_1h = df_1h["Close"].iloc[-1]

        # ENTRY LOGIC
        if not in_position:
            if close_1h > sma_50_1h and rsi_15m < 45:
                return "BUY"

        # EXIT LOGIC
        if in_position:
            if rsi_15m > 70:
                return "SELL"

        return None


Overwriting strategy.py


In [4]:
%%writefile backtest_engine.py
from backtesting import Strategy
import pandas as pd

from indicators import add_indicators_15m, add_indicators_1h
from strategy import MultiTimeframeStrategy


class BacktestStrategy(Strategy):

    def init(self):
        # Single source of truth
        self.logic = MultiTimeframeStrategy()

        # Prepare full dataframe once
        df = self.data.df.copy()

        # 15m indicators
        self.df_15m = add_indicators_15m(df)

        # 1h resample + indicators
        df_1h = df.resample("1H").last().dropna()
        self.df_1h = add_indicators_1h(df_1h)

    def next(self):
        # Slice data up to current index
        current_time = self.data.index[-1]

        df_15m_slice = self.df_15m.loc[:current_time]
        df_1h_slice = self.df_1h.loc[:current_time]

        # Determine position state
        in_position = bool(self.position)

        # Ask strategy for signal
        signal = self.logic.generate_signal(
            df_15m_slice,
            df_1h_slice,
            in_position
        )

        # Execute signal
        if signal == "BUY":
            self.buy(size=0.9)

        elif signal == "SELL" and self.position:
            self.position.close()


Overwriting backtest_engine.py


In [5]:
import yfinance as yf

# Download Bitcoin data (15m intervals for the last 59 days)
# Note: Yahoo Finance only allows 15m data for the last 60 days
data = yf.download("BTC-USD", period="60d", interval="15m")

# Save it so your next code block can find it
data.to_csv("BTCUSDT_15m.csv")

print("File downloaded successfully!")

C:\Users\Hp\AppData\Local\Temp\ipykernel_27872\2382771412.py:5: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download("BTC-USD", period="60d", interval="15m")
[*********************100%***********************]  1 of 1 completed

File downloaded successfully!


In [6]:
import pandas as pd

df = pd.read_csv("BTCUSDT_15m.csv")
print(df.head())
print(df.dtypes)


                       Price          Close           High            Low  \
0                     Ticker        BTC-USD        BTC-USD        BTC-USD   
1                   Datetime            NaN            NaN            NaN   
2  2025-12-01 07:30:00+00:00    86443.78125  86445.4140625   86196.921875   
3  2025-12-01 07:45:00+00:00  86533.8359375  86548.6796875  86478.6484375   
4  2025-12-01 08:00:00+00:00    86457.15625  86635.3515625     86422.1875   

            Open      Volume  
0        BTC-USD     BTC-USD  
1            NaN         NaN  
2   86266.390625  1061277696  
3  86478.6484375  1109180416  
4    86536.78125   742842368  
Price     object
Close     object
High      object
Low       object
Open      object
Volume    object
dtype: object


In [7]:
import pandas as pd

df = pd.read_csv("BTCUSDT_15m.csv", header=None)
print(df.head(5))


                           0              1              2              3  \
0                      Price          Close           High            Low   
1                     Ticker        BTC-USD        BTC-USD        BTC-USD   
2                   Datetime            NaN            NaN            NaN   
3  2025-12-01 07:30:00+00:00    86443.78125  86445.4140625   86196.921875   
4  2025-12-01 07:45:00+00:00  86533.8359375  86548.6796875  86478.6484375   

               4           5  
0           Open      Volume  
1        BTC-USD     BTC-USD  
2            NaN         NaN  
3   86266.390625  1061277696  
4  86478.6484375  1109180416  


In [9]:
df = pd.read_csv("BTCUSDT_15m.csv", header=None)

# Drop the first 2 junk rows
df = df.iloc[3:].copy()

# Assign correct column names
df.columns = ["Datetime", "Close", "High", "Low", "Open", "Volume"]

# Convert datatypes
df["Datetime"] = pd.to_datetime(df["Datetime"], utc=True)

for col in ["Open", "High", "Low", "Close", "Volume"]:
    df[col] = pd.to_numeric(df[col], errors="coerce")

# Set index
df.set_index("Datetime", inplace=True)

print(df.head())
print(df.dtypes)


                                  Close          High           Low  \
Datetime                                                              
2025-12-01 07:30:00+00:00  86443.781250  86445.414062  86196.921875   
2025-12-01 07:45:00+00:00  86533.835938  86548.679688  86478.648438   
2025-12-01 08:00:00+00:00  86457.156250  86635.351562  86422.187500   
2025-12-01 08:15:00+00:00  86594.835938  86594.835938  86466.257812   
2025-12-01 08:30:00+00:00  86741.523438  86741.523438  86553.554688   

                                   Open      Volume  
Datetime                                             
2025-12-01 07:30:00+00:00  86266.390625  1061277696  
2025-12-01 07:45:00+00:00  86478.648438  1109180416  
2025-12-01 08:00:00+00:00  86536.781250   742842368  
2025-12-01 08:15:00+00:00  86466.257812   374222848  
2025-12-01 08:30:00+00:00  86598.570312           0  
Close     float64
High      float64
Low       float64
Open      float64
Volume      int64
dtype: object


In [11]:
import pandas as pd
from backtesting import Backtest

from backtest_engine import BacktestStrategy
from indicators import add_indicators_15m


# --------- Load & clean CSV ---------
def load_data(path):
    df = pd.read_csv(path, header=None)
    df = df.iloc[3:].copy()
    df.columns = ["Datetime", "Close", "High", "Low", "Open", "Volume"]

    df["Datetime"] = pd.to_datetime(df["Datetime"], utc=True)

    for col in ["Open", "High", "Low", "Close", "Volume"]:
        df[col] = pd.to_numeric(df[col], errors="coerce")

    df.set_index("Datetime", inplace=True)
    return df


df = load_data("BTCUSDT_15m.csv")

# --------- Run backtest ---------
bt = Backtest(
    df,
    BacktestStrategy,
    cash=100_000,
    commission=0.0005
)

stats = bt.run()
print(stats)

# --------- Extract trades ---------
trades = stats._trades.copy()

# Build required CSV format
output = pd.DataFrame({
    "timestamp": trades["EntryTime"],
    "symbol": "BTCUSDT",
    "direction": trades["Size"].apply(lambda x: "BUY" if x > 0 else "SELL"),
    "entry_price": trades["EntryPrice"],
    "exit_price": trades["ExitPrice"],
})

output.to_csv("backtest_trades.csv", index=False)

print("\nSaved backtest_trades.csv")
print(output.head())


C:\Users\Hp\AppData\Roaming\Python\Python313\site-packages\backtesting\_plotting.py:55: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support, such as old IDEs. Reset with `backtesting.set_bokeh_output(notebook=False)`.
  warnings.warn('Jupyter Notebook detected. '


Loading BokehJS ...

C:\Users\Hp\Desktop\jupternotebook\25_Jan_2026\backtest_engine.py:21: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  df_1h = df.resample("1H").last().dropna()


Backtest.run:   0%|          | 0/5595 [00:00<?, ?bar/s]

C:\Users\Hp\AppData\Roaming\Python\Python313\site-packages\backtesting\backtesting.py:966: UserWarning: time=198: Broker canceled the relative-sized order due to insufficient margin.
  warnings.warn(
C:\Users\Hp\AppData\Roaming\Python\Python313\site-packages\backtesting\backtesting.py:966: UserWarning: time=212: Broker canceled the relative-sized order due to insufficient margin.
  warnings.warn(
C:\Users\Hp\AppData\Roaming\Python\Python313\site-packages\backtesting\backtesting.py:966: UserWarning: time=219: Broker canceled the relative-sized order due to insufficient margin.
  warnings.warn(
C:\Users\Hp\AppData\Roaming\Python\Python313\site-packages\backtesting\backtesting.py:966: UserWarning: time=220: Broker canceled the relative-sized order due to insufficient margin.
  warnings.warn(
C:\Users\Hp\AppData\Roaming\Python\Python313\site-packages\backtesting\backtesting.py:966: UserWarning: time=223: Broker canceled the relative-sized order due to insufficient margin.
  warnings.warn(


Start                     2025-12-01 07:30...
End                       2026-01-28 14:15...
Duration                     58 days 06:45:00
Exposure Time [%]                    68.12009
Equity Final [$]                 106176.31569
Equity Peak [$]                  111985.22405
Commissions [$]                    2629.90306
Return [%]                            6.17632
Buy & Hold Return [%]                 3.99663
Return (Ann.) [%]                    44.88336
Volatility (Ann.) [%]                32.12203
CAGR [%]                             45.54734
Sharpe Ratio                          1.39728
Sortino Ratio                         2.73678
Calmar Ratio                          5.28807
Alpha [%]                             4.35319
Beta                                  0.45617
Max. Drawdown [%]                    -8.48766
Avg. Drawdown [%]                    -1.10732
Max. Drawdown Duration       13 days 00:15:00
Avg. Drawdown Duration        1 days 09:53:00
# Trades                          

In [12]:
pip install python-binance ta


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [13]:
%%writefile live_trader.py
import time
import pandas as pd
from binance.client import Client

from indicators import add_indicators_15m, add_indicators_1h
from strategy import MultiTimeframeStrategy

# ---------------- CONFIG ----------------
API_KEY = "Uyocz2D4FbMw1Jw4nguXFvtjE2BzChRgutWsOQmCOVJd65xR7Iw0d98S6ORCyfs5"
API_SECRET = "mAloxT3mFuvUaGn6HOtWuavGQvoeNdSK8T3ySoMFLiehYJFXQRUNBPUQHM6CwYQd"

SYMBOL = "BTCUSDT"
INTERVAL_15M = Client.KLINE_INTERVAL_15MINUTE
LOOKBACK_LIMIT = 200

# ---------------- CLIENT ----------------
client = Client(API_KEY, API_SECRET)
client.API_URL = "https://testnet.binance.vision/api"

# ---------------- STRATEGY ----------------
strategy = MultiTimeframeStrategy()

# ---------------- STATE ----------------
in_position = False
entry_price = None

# ---------------- CSV INIT ----------------
csv_file = "live_trades.csv"

if not pd.io.common.file_exists(csv_file):
    pd.DataFrame(
        columns=["timestamp", "symbol", "direction", "entry_price", "exit_price"]
    ).to_csv(csv_file, index=False)

# ---------------- HELPERS ----------------
def fetch_15m_data():
    klines = client.get_klines(
        symbol=SYMBOL,
        interval=INTERVAL_15M,
        limit=LOOKBACK_LIMIT
    )

    df = pd.DataFrame(klines, columns=[
        "OpenTime", "Open", "High", "Low", "Close", "Volume",
        "_", "_", "_", "_", "_", "_"
    ])

    df["Datetime"] = pd.to_datetime(df["OpenTime"], unit="ms", utc=True)
    df = df[["Datetime", "Open", "High", "Low", "Close", "Volume"]]

    for col in ["Open", "High", "Low", "Close", "Volume"]:
        df[col] = pd.to_numeric(df[col])

    df.set_index("Datetime", inplace=True)
    return df


# ---------------- MAIN LOOP ----------------
while True:
    try:
        df_15m = fetch_15m_data()
        df_15m = add_indicators_15m(df_15m)

        df_1h = df_15m.resample("1H").last().dropna()
        df_1h = add_indicators_1h(df_1h)

        signal = strategy.generate_signal(
            df_15m,
            df_1h,
            in_position
        )

        if signal == "BUY" and not in_position:
            order = client.create_order(
                symbol=SYMBOL,
                side="BUY",
                type="MARKET",
                quantity=0.001
            )

            entry_price = float(order["fills"][0]["price"])
            in_position = True

            print(f"BUY @ {entry_price}")

        elif signal == "SELL" and in_position:
            order = client.create_order(
                symbol=SYMBOL,
                side="SELL",
                type="MARKET",
                quantity=0.001
            )

            exit_price = float(order["fills"][0]["price"])
            in_position = False

            trade = {
                "timestamp": pd.Timestamp.utcnow(),
                "symbol": SYMBOL,
                "direction": "BUY",
                "entry_price": entry_price,
                "exit_price": exit_price
            }

            pd.DataFrame([trade]).to_csv(
                csv_file,
                mode="a",
                header=False,
                index=False
            )

            print(f"SELL @ {exit_price}")

        time.sleep(60)

    except Exception as e:
        print("Error:", e)
        time.sleep(60)


Writing live_trader.py


In [ ]:
%run live_trader.py

C:\Users\Hp\Desktop\jupternotebook\25_Jan_2026\live_trader.py:64: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  df_1h = df_15m.resample("1H").last().dropna()
